In [ ]:
from centrex_tlf_hamiltonian import states, hamiltonian, transitions
import centrex_tlf_couplings as couplings
import centrex_tlf_lindblad as lindblad
import numpy as np
from scipy import constants as cst

import matplotlib.pyplot as plt

In [ ]:
plt.rcParams.update({'font.size': 15})

In [ ]:
def doppler_shift(velocity, frequency):
    """calculate the doppler shifted frequency
    Args:
        velocity (float): velocity [m/s]
        frequency (float): frequency [Hz]
    Returns:
        float: doppler shifted frequency in Hz
    """
    return frequency * (1 + velocity / cst.c)


def velocity_to_detuning(velocity, frequency=1.103e15, Γ=2*np.pi*1.56e6):
    """convert velocity to detuning in units of Γ
    Args:
        velocity (float): velocity [m/s]
        frequency (float): frequency [Hz]
        Γ (float): Γ [2π⋅Hz]
    Returns:
        float: detuning in units of Γ
    """
    return (doppler_shift(velocity, frequency) - frequency) * 2 * np.pi / Γ

In [ ]:
trans = [transitions.OpticalTransition(transitions.OpticalTransitionType.R, J_ground=2, F1=7/2, F=4)]

polarizations = [
    [couplings.polarization_σp]
]

transition_selectors = couplings.generate_transition_selectors(
    trans, polarizations, 
    ground_mains = [
        1*states.CoupledBasisState(
            J=2, F=3, F1=5/2, mF=0, I1=1/2, I2=1/2, Ω=0, P=+1, 
            electronic_state=states.ElectronicState.X
            )
        ],
    excited_mains=[
        1*states.CoupledBasisState(
            J=3, F=4, F1=7/2, mF=1, I1=1/2, I2=1/2, Ω=1, P=-1, 
            electronic_state=states.ElectronicState.B
        )
    ]
)

In [ ]:
v_to_Γ = velocity_to_detuning(1)*hamiltonian.Γ


odepars = lindblad.odeParameters(
    Ω0 = "Ωl0 * phase_modulation(t, β, ωphase)",
    Ωl0     = 1*hamiltonian.Γ,    # Rabi frequency of the laser [rad/s]
    δ0      = f"vx*{v_to_Γ}", # detuning of the laser [rad/s]
    
    # laser phase modulation
    ωphase = hamiltonian.Γ,       # laser phase modulation frequency [rad/s]
    β      = 3.8,             # laser phase modulation depth [rad]
    
    Pσp0 = 1,

    # molecules
    z0 = 0,                   # molecule start z position [m]
    vz = 184,                 # longitudinal molecular velocity [m/s]
    vx = 0,
)

In [ ]:
%%time
obe_system = lindblad.setup_OBE_system_julia_transitions(
    odepars, trans, transition_selectors, verbose=True, full_output=True, 
    qn_compact= True,
    decay_channels = None,
    E = np.array([0,0,200])
)

In [ ]:
couplings.generate_br_dataframe(obe_system.ground, obe_system.excited, group_ground='J')

In [ ]:
obe_system.couplings[0]

In [ ]:
obe_system.H_symbolic

In [ ]:
qn_select = states.QuantumSelector(J=2, F=3, electronic = states.ElectronicState.X)
indices = qn_select.get_indices(obe_system.QN)
ρ = np.zeros(obe_system.H_symbolic.shape, dtype = complex)
for idx in indices:
    ρ[idx,idx] = 1 / indices.size

In [ ]:
problem = lindblad.OBEProblem(odepars, ρ, tspan = (0,200e-6))
config = lindblad.OBEProblemConfig(saveat = 1e-6)

In [ ]:
results = lindblad.do_simulation_single(problem)

In [ ]:
qn_select_excited = states.QuantumSelector(electronic=states.ElectronicState.B)
indices_excited = qn_select_excited.get_indices(obe_system.QN)

In [ ]:
fig, ax = plt.subplots(figsize = (10,6)) 
ax.plot(results.t*odepars.vz, results.y.T, lw = 2)
ax.set_xlabel('distance [m]')
ax.set_ylabel('population')
# ax.legend(fontsize = 14)
ax.grid(True)

nphotons = np.trapz(results.y[indices_excited], x = results.t).sum() * hamiltonian.Γ
print(f"{nphotons:.2f} photons")

In [ ]:
print(obe_system.couplings[0].fields[0].field.shape)
print(len(obe_system.QN))

In [ ]:
obe_system.couplings

In [ ]:
from typing import List
def generate_qn_compact(
    transitions: List[couplings.TransitionSelector],
    X_basis: List[states.State],
):
    J_transitions_ground = []
    for transition in transitions:
        J_transitions_ground.append(transition.J_ground)
    J_compact = [
        Ji
        for Ji in np.unique([s.largest.J for s in X_basis])  # type: ignore
        if Ji not in J_transitions_ground
    ]
    qn_compact = [
        states.QuantumSelector(J=Ji, electronic=states.ElectronicState.X)
        for Ji in J_compact
    ]

    return qn_compact